# ООП (Продолжение)

## Сюжет 1: Геометрические фигуры

Ещё раз про "базовое ООП": инкапсуляция ("наполнение" класса разным), наследование (один класс — частный случай другого), полиморфизм (имена методов одинаковые, но "начинка" разная).
Плюс (ещё) немного абстракции.

In [1]:
PI = 3.14


class GeometricFigure:
    # Абстрактный метод — пока его нет,
    # но у всех "нормальных" потомков он должен быть.
    def describe(self):
        raise NotImplementedError()
    
    # Класс с абстрактным методом — абстрактный класс

    def area(self):
        raise NotImplementedError()


class Polygon(GeometricFigure):
    def __init__(self, vertices):
        self.vertices = vertices
    
    def describe(self):
        return f'Polygon with {len(self.vertices)} vertices'
    
    def perimeter(self):
        if len(self.vertices) == 1:
            return 0
        elif len(self.vertices) == 2:
            return self._calc_distance(
                self.vertices[0], self.vertices[-1]
            )
        
        p = 0
        
        for i in range(len(self.vertices)):  # i = 0 тоже ОК (замкнутая фигура)
            p += self._calc_distance(
                self.vertices[i], self.vertices[i - 1]
            )
        
        return p
    
    def _calc_distance(self, p1, p2):
        return ((p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2) ** 0.5
    
    # Без метода area класс Многоугольника тоже получается абстрактным


class Rectangle(Polygon):
    def __init__(self, x, y, w, h):
        super().__init__(
            vertices=[
                (x, y), (x + w, y),
                (x + w, y + h), (x, y + h)
            ]
        )
        self.width = w
        self.height = h
    
    def area(self):
        return self.width * self.height


class Square(Rectangle):
    def __init__(self, x, y, a):
        super().__init__(x, y, a, a)


class Ellipse(GeometricFigure):
    def __init__(self, x, y, a, b):
        self.x = x
        self.y = y
        self.a = max(a, b)
        self.b = min(a, b)
    
    def describe(self):
        return f'Ellipse with semi-major axis {self.a} and semi-minor axis {self.b}.'

    def area(self):
        return PI * self.a * self.b


class Circle(Ellipse):
    def __init__(self, x, y, r):
        super().__init__(x, y, r, r)

In [2]:
polygon = Polygon([(1, 0), (0, 1), (0.5, 0.5)])

print(polygon.describe())
print(polygon.vertices)

Polygon with 3 vertices
[(1, 0), (0, 1), (0.5, 0.5)]


In [3]:
# Абстрактный (не до конца готовый) класс,
# поэтому в процессе работы можно "наступить на грабли"
polygon.area()

NotImplementedError: 

In [4]:
rectange = Rectangle(0, 0, 1, 1)

print(rectange.describe())
print(rectange.vertices)
print(f'Area: {rectange.area()}')
print(f'Perimeter: {rectange.perimeter()}')

Polygon with 4 vertices
[(0, 0), (1, 0), (1, 1), (0, 1)]
Area: 1
Perimeter: 4.0


In [5]:
square = Square(0, 0, 1)

print(square.describe())
print(square.vertices)
print(f'Area: {square.area()}')
print(f'Perimeter: {square.perimeter()}')

Polygon with 4 vertices
[(0, 0), (1, 0), (1, 1), (0, 1)]
Area: 1
Perimeter: 4.0


In [6]:
ellipse = Ellipse(0, 0, 1, 2)

print(ellipse.describe())
print(f'Area: {ellipse.area()}')

Ellipse with semi-major axis 2 and semi-minor axis 1.
Area: 6.28


In [7]:
circle = Circle(0, 0, 1)

print(circle.describe())
print(f'Area: {circle.area()}')

Ellipse with semi-major axis 1 and semi-minor axis 1.
Area: 3.14


## Сюжет 2: Векторы

Про "магические" возможности классов, или ещё несколько методов со "страшными" именами (кроме конструктора).

In [8]:
v1 = (1, 2)
v2 = (-1, -1)

In [9]:
# Не то, что ожидаем от сложения векторов
v1 + v2

(1, 2, -1, -1)

In [10]:
# Тоже не то
v1 * 2

(1, 2, 1, 2)

Хочется, чтобы с векторами было удобно работать...

In [11]:
class Vector:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def __add__(self, other):
        if not isinstance(other, Vector):
            # Так принято делать в Питоне, если операцию провести нельзя.
            # (По идее тут надо бы было бросить NotImplementedError,
            # но так не делают, Питон сам бросит ошибку; https://stackoverflow.com/questions/878943/why-return-notimplemented-instead-of-raising-notimplementederror)
            return NotImplemented

        return Vector(
            x = self.x + other.x,
            y = self.y + other.y
        )
    
    def __mul__(self, other):
        if not isinstance(other, (int, float)):
            return NotImplemented
        
        print('"Left" mul.')

        return Vector(
            x = self.x * other,
            y = self.y * other
        )
    
    def __sub__(self, other):
        return self + (-1) * other
    
    def __rmul__(self, other):
        print('"Right" mul.')

        return self * other
    
    def __abs__(self):
        return (self.x ** 2 + self.y ** 2) ** 0.5

    def __str__(self):  # "Человекочитаемое" описание
        return f'({self.x}, {self.y})'
    
    def __repr__(self):  # "Машинопонятное" ("программистское") описание
        return f'Vector({self.x}, {self.y})'

In [12]:
v1 = Vector(3, 0)
v2 = Vector(0, 4)

In [13]:
v1.x, v1.y

(3, 0)

In [14]:
print(v1 + v2)

(3, 4)


In [15]:
print(v1 * 2)

"Left" mul.
(6, 0)


In [16]:
print(2 * v1)

"Right" mul.
"Left" mul.
(6, 0)


In [17]:
print(v1 * v2)

TypeError: unsupported operand type(s) for *: 'Vector' and 'Vector'

In [18]:
print(v1 + 2)

TypeError: unsupported operand type(s) for +: 'Vector' and 'int'

In [19]:
v = v1 + v2

In [20]:
# Функции "из ниоткуда" Питона используют магические методы,
# определённые "где-то там под капотом"
str(v)

'(3, 4)'

In [21]:
abs(v)

5.0